In [15]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
import scipy.sparse
import scipy.sparse.linalg


In [16]:
v, f = igl.read_triangle_mesh('data/hand.off')
labels = np.load('data/hand.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()
mp.plot(v, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

In [17]:
# labels = np.load("data/woody-hi.label.npy")
# v, f = igl.read_triangle_mesh("data/woody-hi.off")
# p = mp.plot(v, f, c=labels)
# v -= v.min(axis=0)
# v /= v.max()

In [18]:
# COMPUTING B: 
handles_indices = np.where(labels > 0)[0]
free_indices = np.where(labels == 0)[0]

L = igl.cotmatrix(v, f)
M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
Minv_diag = 1 / M.diagonal()
Minv = np.diag(Minv_diag)

# bi-Laplacian
Q2 = Minv @ L @ L
Q2_free = Q2[free_indices, :][:, free_indices]
Q2_mixed = Q2[free_indices, :][:, handles_indices]
rhs = -Q2_mixed @ v[handles_indices]

v_smooth = v.copy()
for i in range(3): #x,y,z
    v_smooth[free_indices, i]    = np.linalg.solve(Q2_free, rhs[:, i])
    v_smooth[handles_indices, i] = v[handles_indices, i]

B = v_smooth.copy()
p = mp.plot(v_smooth, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

In [19]:
# COMPUTING B'(Bc):
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    Bc = target_pos.copy()
    free = free_indices
    handles = handles_indices
    
    rhs = -Q2_mixed @ Bc[handles]

    for i in range(3): #x,y,z
        Bc[free, i] = np.linalg.solve(Q2_free, rhs[:, i])
        Bc[handles, i] = target_pos[handles_indices, i]

    return Bc
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''
Bc = position_deformer(B)
mp.plot(Bc, f, c=labels)
# pos_f.deformer = position_deformer

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

In [20]:
handle_vertex_positions = B.copy()
# handle_vertex_positions = v.copy()

pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = B[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print("v.shape:", v.shape)
    print("labels.shape:", labels.shape)
    print("slices shape:", slices.shape)
    print("np.sum(slices):", np.sum(slices))

    print('FPS', 1/(t1 - t0))
# pos_f.deformer = lambda x:x
pos_f.deformer = position_deformer

In [21]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [22]:
## Widget UI

p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4), value=1), FloatSlider(value=0.0, descrip…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

In [23]:
# compute orthogonal reference frame on every vertex v of B 

# (1) unit vertex normal
n = igl.per_vertex_normals(B, f) 

# (2)
VF = [[] for _ in range(len(v))]
for face in f:
    for i in range(3):
        a = face[i]
        b = face[(i + 1) % 3]
        VF[a].append(b)
        VF[b].append(a)

t = np.zeros_like(B)
used_edge = np.zeros(len(B), dtype=int)   
for i in range(len(v)):
    vi = B[i]
    ni = n[i]
    max_proj = 0
    best_vec = None
    best_j = -1
    for j, nei in enumerate(VF[i]):
        edge = B[nei] - vi
        proj = edge - np.dot(edge, ni) * ni  # v project onto tangent plane
        length = np.linalg.norm(proj)
        if length > max_proj: # finding longest projection
            max_proj = length
            best_vec = proj / length
            best_j = nei
    # print(max_proj)
    if best_vec is not None:
        t[i] = best_vec
        used_edge[i] = best_j
    else:
        t[i] = np.array([1.0, 0.0, 0.0]) 

b = np.cross(n, t) # Cross Product of (1) and (2)



In [24]:
tB, bB, nB = t, b, n
fB = np.stack([tB, bB, nB], axis=2) # n, 3, 3 

# displacements from v to B in B's frame
dB = np.zeros_like(v)
for i in range(len(v)):
    disp = v[i] - B[i]
    dB[i] = fB[i].T @ disp

# Bc reference frames using same edges
nBc = igl.per_vertex_normals(Bc, f)
tBc = np.zeros_like(v)
for i in range(len(v)):
    if used_edge[i] == -1:
        tBc[i] = tB[i]
    edge = Bc[used_edge[i]] - Bc[i]
    projection = edge - np.dot(edge, nBc[i]) * nBc[i]
    norm = np.linalg.norm(projection)
    if norm > 1e-6:
        tBc[i] = projection / norm  
    else:
        tBc[i] = tB[i]

bBc = np.cross(nBc, tBc)
fBc = np.stack([tBc,bBc,nBc], axis = 2)

# Bc -> S'
Sc = np.zeros_like(v)
for i in range(len(v)):
    Sc[i] = Bc[i] +fBc[i] @ dB[i]

# print(np.linalg.norm(fBc[:, 0], axis=1))  
# print(np.linalg.norm(fBc[:, 1], axis=1))  
# print(np.linalg.norm(fBc[:, 2], axis=1)) 




In [25]:
# B with displacements
p1 = mp.plot(B, f, c=labels)
p1.add_lines(B, v, shading={"line_color": "red"}) 


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

1

In [26]:
# B' with displacements
p2 = mp.plot(Bc, f, c=labels)
p2.add_lines(Bc, Sc, shading={"line_color": "green"}) 

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

1

In [27]:
## Widget UI
p_Sc = mp.plot(handle_vertex_positions, f, c=labels)
def pos_f_Sc(s, x, y, z, α, β, γ):
    slices = (labels == s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = B[slices] + np.array([[x, y, z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x, y, z, α, β, γ]

    Bc = pos_f_Sc.deformer(handle_vertex_positions)
    
    # Bc reference frames using same edges
    nBc = igl.per_vertex_normals(Bc, f)
    tBc = np.zeros_like(v)
    for i in range(len(v)):
        if used_edge[i] == -1:
            tBc[i] = tB[i]
        edge = Bc[used_edge[i]] - Bc[i]
        projection = edge - np.dot(edge, nBc[i]) * nBc[i]
        norm = np.linalg.norm(projection)
        if norm > 1e-6:
            tBc[i] = projection / norm  
        else:
            tBc[i] = tB[i]

    bBc = np.cross(nBc, tBc)
    fBc = np.stack([tBc,bBc,nBc], axis = 2)

    # Bc -> S'
    Sc = np.zeros_like(v)
    for i in range(len(v)):
        Sc[i] = Bc[i] +fBc[i] @ dB[i]
    p_Sc.update_object(vertices=Sc)


pos_f_Sc.deformer = position_deformer
iw.interact(pos_f_Sc, **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4), value=1), FloatSlider(value=0.0, descrip…

<function __main__.pos_f_Sc(s, x, y, z, α, β, γ)>